In [1]:

import tweepy as tw
import json
import pandas as pd
import csv
import geocoder
# self creaated functions in the script : create_hashtag_text. Takes a list of tweet objects as parameter and extracts the text of hashtags
# Returns another list which contains sublists of hashtags. 

In [3]:
# meine credentials!
twitter_keys = {
        'consumer_key' : '',
        'consumer_secret' : '',
        'access_token_key' : '',
        'access_token_secret' : ''
    
}

In [4]:
# setting up access to API

auth = tw.OAuthHandler(twitter_keys['consumer_key'], twitter_keys['consumer_secret'])
auth.set_access_token(twitter_keys['access_token_key'], twitter_keys['access_token_secret'])

api = tw.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

In [ ]:
# testing API to see if it works, it works

# api.update_status("Posting my first tweet through Python! #tweepy #thisiscool")

   <h3> Explanation </h3>

- search tweets by specific hashtag and store them in tweets variable. 
- __[Read up on tweepy cursor documentation](http://docs.tweepy.org/en/latest/cursor_tutorial.html)__

 1. Use cursor object to search tweets including a specifif hashtag(or even a word.)
 2. Store the returned object in a variable.
 3. *(Optional)* Convert into a better format or work with the object as is.
 4. Extract data into a framework and then save to csv file.
 ---
   - **Note: since_id for date uses YYYY-MM-DD format. I was stuck on this for an hour :(**


 ---
 

In [ ]:
"""

In [5]:
# define search text and date from which to scrape tweets 
# (Since we are using standard twitter API, only tweets 
# from last 2 weeks from the time of running code us availabe.)

search_word = input("Enter a word/ hashtag you'd like to search. \
                     In-case of hashtag, include # in parameter\n")

if search_word == "":
    search_word = "#tweepy" + " -filter:retweets"

else :
    search_word = search_word + " -filter:retweets"
    
date_since = "2020-08-01"
    
print(search_word,"\t", date_since, '\n')


results = [tweet for tweet in tw.Cursor(api.search, q = search_word,
                                        lang = 'en', since = date_since,
                                        tweet_mode = 'extended').items(200)]


print(type(results))
print(len(results))

#RamMandir -filter:retweets 	 2020-08-01 

<class 'list'>
200


# Function created to extract just the text from entities.get('hashtags') object. #

## Context for problem encountered: 
   - The entities object is a tweet object with a 'hashtags' key.
  - The 'hashtags' value is a dictionary, inside of which is a list, which contains dictionary(ies) which have a text and a indices key.
   - Multiple hashtags in a tweet results in multiples dictionaries each containing a 'text' and 'indices' key
   - Example of hashtag object: 
                                     ```{
                                    "hashtags": [
                                      {
                                        "indices": [
                                          32,
                                          38
                                        ],
                                        "text": "nodejs"
                                      }
                                    ]
                                  }```
   
   - Extracting it using just one line was elusive to me, still is :(
   - Hence the need for this function.          

### Basic logic of function: ###
1. Put the object returned from entities.get('hashtags') into a list(hash_list) one at a time
  - This will create an main outer list, it acts as the container, each sublist inside it corresponds to one tweet and the dictionaries in        the sublist correspond to hashtags in that tweet.
2. Then traverse the hash_list(outerList) using nested for loop.
3. First for loop will iterate one by one through the object, the inner for loop will use .get() method to extract the 'text' value from it.
    - Initialize 2 empty list, text list and temp list.
    - in the 1st for loop, add condition, ```**if the temp_list in not empty, we will append it to text_list.**```
    - initialize the temp_list as empty again(reasons below)
    - for nested(2nd) for loop, iterate and append to temp_list values of dcitionary using .get('text') method. 
        > Example, if a tweet has 3 hashtags, the temp_list will have 3 entries.
    - The temp_list (if not empty), will be appended to the text_list.
    - to store the final value of temp_ list, we will put the text_list.append(temp_list) out of both the loop.
```
def create_hashtag_text(results):

    hash = []
    for tweet in results[:10]:
        hash.append(tweet.entities.get('hashtags'))

    text_list = []
    # list of hashtags corresponsing to that tweet
    temp_list = []
    # temporary empty list

    for sublist in hash[:3]:
        # we had a bug here, since  we initialize the templist for every outerloop, it works the the first time we run the code
        # but the next time  we  run the code, text_list list is appended with previous run's templist variables thus appending it at the very 
        # beginning. Solved it  by initializing temp_list outside the loop and then once again inside the outerloop. THis created another
        # problem where first element in t_hash list was an empty list, fixed that by adding a if block that would only allow temp_list to
        # be appended if temp_list is not empty. 
        if temp_list:
            text_list.append(temp_list)
        print(sublist)
        temp_list = list()
        i = 1
        for item in sublist:
            print(item.get('text'),'\t')
            temp_list.append(item.get('text'))
            temp_list.append(i)
            i +=1
    text_list.append(temp_list)

    return text_list.
```




In [120]:
def create_hashtag_text(results):

    """ This function takes a tweet object and returns a list of hashtags.

    Args:
    results ([tweet object]): is a nested tweet object

    Returns:
    [list]: Is a list of list of hashtags for every tweet.
    """
    
    hash = []
    # empty list that will contain all hashtag objects.
    for item in results:
        thing = item.entities.get('hashtags')
        # print(type(thing)) #it's a list
        if thing:
            # if in case the tweet had no hashtag, return a list with dictionary
            # giving 404 error 
            hash.append(thing)
        else:
            hash.append([{'text': 404}])

    text_list = []
    # list of hashtags corresponsing to that tweet
    temp_list = []
    # temporary empty list

    for sublist in hash:

        if temp_list:
            text_list.append(temp_list)
        # print(sublist)
        temp_list = list()
        i = 1
        for item in sublist:
            # print(item.get('text'), '\t')
            temp_list.append(item.get('text'))
            temp_list.append(i)
            i += 1
    text_list.append(temp_list)

    return text_list


In [121]:
# store tweets in a DataFrame

def tweets_info(list):

    # list comprehension being used

    id_list = [tweet.id for tweet  in list]
    data_set = pd.DataFrame(id_list, columns = ["id"])

    # pandas DataFrame have similar syntax to Dictionary DS

    data_set["place"] = [(tweet.place.full_name,',',tweet.place.country) if tweet.place is not None 
                         else '' for tweet in list]
    # except:
    #     data_set["place"] = ['Null' for tweet in list]

    data_set["text"] = [tweet.full_text for tweet in list]
    data_set["created_at"] = [tweet.created_at for tweet in list]
    # data_set["retweet_count"] = [tweet.retweet_count for tweet in list]
    data_set["user_screen_name"] = [tweet.user.screen_name for tweet in list]
    # data_set["user_followers_count"] = [tweet.author.followers_count for tweet in list]
    data_set["user_location"] = [tweet.user.location for tweet in list]
    data_set["Hashtags"] = [tweet.entities.get('hashtags') for tweet in list]
    
    return data_set

data_set = tweets_info(results)
hash_text = create_hashtag_text(results)
# creating a list of hashtag texts.
data_set["Hash_text"] = [text_item for text_item in hash_text]



In [122]:
print(len(results))
print(len(create_hashtag_text(results)))

200
200


In [ ]:
# writing dataframe to csv format
data_set.to_csv("")